In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from io import BytesIO
import os
import sys
import PIL.Image as PILImage
import requests
import random
import objaverse
from termcolor import colored

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
from src import *
objaverse._VERSIONED_PATH=os.path.join('../.objaverse', "hf-objaverse-v1")

# Generate the dataset
##### ℹ️ This notebook requires `data/2-annotations_filtered_by_thumbnails.parquet` and `statistics.parquet`, which is generated by `compute_statistics.py`
This notebook generate the dataset as triplets of _rendering_, _UV map_ and _diffuse texture_.
##### ⚠️ This notebook is just a preview of the actual script (see [generate_dataset.py](generate_dataset.py)). The generation of the entire dataset is expensive and thus requires a computation node.

In [2]:
annotations = pd.read_parquet('../data/2-annotations_filtered_by_thumbnails.parquet')
statistics = pd.read_parquet('statistics.parquet')

Select only the objects with 1 Mesh, 1 UV map and 1 Diffuse texture

In [ ]:
uids = statistics[statistics["diffuseCount"] == 1].index
gbls = objaverse.load_objects(annotations.index[:40_000].to_list(), download_processes=32)

In [5]:
MIN_UV_DENSITY = 0.01
SAMPLES = 5
random.seed(42)

for uid in uids[random.sample(range(len(uids)), SAMPLES)]:
    path = os.path.join(gbls[uid], "models", "model_normalized.obj")
    obj = ObjaverseObject3D(uid, path)

    # Get the thumbnail
    thumbnail = requests.get(annotations.loc[uid]["thumbnail"]).content
    render = PILImage.open(BytesIO(thumbnail))

    # Extract diffuse texture
    mesh = next(x for x in load_model(gbls[uid]) if x.type == "MESH")
    diffuse = obj.textures[0]

    # Bake UV map
    uv_map = obj.draw_uv_map()

    plot_images([render, diffuse, uv_map], cols=3, size=5)
    uv_density = compute_opacity(uv_map)
    print(
        colored(
            f"UV density = {uv_density:.2%} --> {'OK' if uv_density > MIN_UV_DENSITY else 'TOO LOW'}",
            "green" if uv_density > MIN_UV_DENSITY else "red",
        )
    )

ERROR (io.obj): source/blender/io/wavefront_obj/importer/obj_import_file_reader.cc:435 OBJParser: Cannot read from OBJ file:'../.objaverse/hf-objaverse-v1/glbs/000-047/44455efadbdd4e3b8bb66623d6d5361a.glb/models/model_normalized.obj'.

OBJ import of 'model_normalized.obj' took Error: OBJ Import: Cannot open file '../.objaverse/hf-objaverse-v1/glbs/000-047/44455efadbdd4e3b8bb66623d6d5361a.glb/models/model_normalized.obj'
1.39 ms


RuntimeError: Error: OBJ Import: Cannot open file '../.objaverse/hf-objaverse-v1/glbs/000-047/44455efadbdd4e3b8bb66623d6d5361a.glb/models/model_normalized.obj'
